In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import optuna

In [25]:
# 設定値
class CFG:
    seed = 3407 #52
    n_folds = 10 #replaced 20
    target_col = 'Class'
    metric = 'balanced_log_loss'
    metric_maximize_flag = False
    num_boost_round = 50500
    early_stopping_round = 100
    verbose = 2000
    boosting_type = 'dart'
    
    # light-gbm設定値
    lgb_params = {
        'objective': 'binary', # 'binary', 'multiclass'
        'metric': None, # 'auc', 'multi_logloss'
        # 'num_class': None,
        'boosting': boosting_type,
        # 'device_type':'gpu',
        'learning_rate': 0.005,
        'num_leaves': 5,
        'feature_fraction': 0.50,
        'bagging_fraction': 0.80,
        'lambda_l1': 2, 
        'lambda_l2': 4,
        'n_jobs': -1,
        'is_unbalance':True, #added balancing
        'verbose': -1, #added silence
        # 'min_data_in_leaf': 40,
        # 'bagging_freq': 10,
        'seed': seed,
    }

In [26]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
test_df[CFG.target_col] = -1
submission_df = pd.read_csv("../data/sample_submission.csv")
all_df = pd.concat([train_df, test_df])

In [4]:
test_df

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1


In [5]:
submission_df

,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


BC, CLはいらん

In [15]:
numerical_features = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL']
categorical_features = ['EJ']
features = numerical_features + categorical_features

### balanced loglossの計算（学習で使う？）

## 各種パラメータの意味
### objective
* 回帰ならregression
* ランク学習ならlambdarank
### metric
* メトリクスとして何を使うか
* ndcg, mapなどを指定できる
### ndcg/map_eval_at
* NDCG/MAP@kのkを設定する
* [12, 24]など複数指定も可能
### num_leaves
* 決定木の最大の葉の数
* 大きいほど複雑なモデルを作れるが、過学習になりやすい
* default: 31
### learning_rate
* 学習率
* 学習するたびにどれくらいその結果を強く反映させるか
* 小さいほど細かなチューニングができるが、時間がかかる
* default: 0.1
### feature_fraction
* Scikit-Learn　APIでの名前: colsample_bytree
* 決定木ごとにランダムに抽出される列の割合
* 値が大きいほど過学習になりやすい
* default: 1.0
### bagging_freq
* 設定した値のイテレーション毎にバギング実施する
* 大きいほど過学習になりやすい
* default: 0
### bagging_fraction
* Scikit-Learn　APIでの名前: subsample
* 決定木ごとにランダムに抽出される行（標本）の割合
* 値が大きいほど過学習になりやすい
* default: 1.0
### random_state
* default: None

In [27]:
# 前処理
def Preprocessing(input_df: pd.DataFrame)->pd.DataFrame:
    output_df = input_df.copy()
    output_df['EJ'] = input_df['EJ'].replace({'A': 0, 'B': 1})
    return output_df

all_df = Preprocessing(all_df)

train_df = all_df[all_df[CFG.target_col] != -1].copy()
test_df = all_df[all_df[CFG.target_col] == -1].copy()

In [28]:
# 評価基準
def balanced_log_loss(y_true, y_pred):
    # 予測値を任意の最大値、最小値に抑える
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    # y_trueの頻度をカウント
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    balanced_log_loss_score = (-w0/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(1-y_pred))) - w1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred)))) / (w0+w1)
    return balanced_log_loss_score

# lgb用の評価指標
def lgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), CFG.metric_maximize_flag

# balanced _log_loss の重み計算
def calc_log_loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

In [29]:
def objective(trial):
    lgb_params = {
        "objective": "binary",
        "learning_rate"=0.01, 
        "metric": "binary_logloss",
        "verbosity": -1, # 学習途中の情報を表示するかどうか
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    scores = []
    # K-分割交差検証(層化抽出法)
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df, train_df[CFG.target_col])):
        # 進行状況を出力
        print('-'*50)
        print(f' training fold {fold + 1}')
        
        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        
        # 訓練データの重みを計算
        train_w0, train_w1 = calc_log_loss_weight(y_train)
        # 検証データの重みを計算
        valid_w0, valid_w1 = calc_log_loss_weight(y_valid)
        # 訓練データをlgb用に変換
        lgb_train = lgb.Dataset(x_train, y_train, weight=y_train.map({0: train_w0, 1: train_w1}), categorical_feature=categorical_features)
        # 検証データをlgb用に変換
        lgb_valid = lgb.Dataset(x_valid, y_valid, weight=y_valid.map({0: valid_w0, 1: valid_w1}), categorical_feature=categorical_features)
        model = lgb.train(
                    params = lgb_params,
                    train_set = lgb_train,
                    num_boost_round = CFG.num_boost_round,
                    valid_sets = [lgb_train, lgb_valid],
                    early_stopping_rounds = CFG.early_stopping_round,
                    verbose_eval = CFG.verbose,
                    # feval = lgb_metric,
                )
        # 予測
        preds = model.predict(x_valid)
        # 予測値をラベルに変換
        pred_labels = np.rint(preds)
        # 評価
        val_score = balanced_log_loss(y_valid, pred_labels)
        
        scores.append(val_score)
    # クロスバリデーションの平均値を計算
    mean_score = np.mean(scores)
    return mean_score

In [32]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2023-05-24 19:20:46,291] A new study created in memory with name: no-name-6ec39d3a-f069-4ce7-998c-20010f511bae
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be r

--------------------------------------------------
 training fold 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[72]	training's binary_logloss: 0.0475253	valid_1's binary_logloss: 0.353475
--------------------------------------------------
 training fold 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[121]	training's binary_logloss: 0.0168361	valid_1's binary_logloss: 0.132018
--------------------------------------------------
 training fold 3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[81]	training's binary_logloss: 0.0342991	valid_1's binary_logloss: 0.218777
--------------------------------------------------
 training fold 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	training's binary_logloss: 0.14123	valid_1's binary_logloss: 0.406458
---------------------------------

/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.

Early stopping, best iteration is:
[88]	training's binary_logloss: 0.0300489	valid_1's binary_logloss: 0.218334
--------------------------------------------------
 training fold 6
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[84]	training's binary_logloss: 0.0327472	valid_1's binary_logloss: 0.191056
--------------------------------------------------
 training fold 7
Training until validation scores don't improve for 100 rounds


/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.

...

Early stopping, best iteration is:
[184]	training's binary_logloss: 0.0911718	valid_1's binary_logloss: 0.129513
--------------------------------------------------
 training fold 8
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[188]	training's binary_logloss: 0.0835751	valid_1's binary_logloss: 0.187793
--------------------------------------------------
 training fold 9
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	training's binary_logloss: 0.187497	valid_1's binary_logloss: 0.319492
--------------------------------------------------
 training fold 10
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[97]	training's binary_logloss: 0.159927	valid_1's binary_logloss: 0.202088
--------------------------------------------------
 training fold 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration 

/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[232]	training's binary_logloss: 0.084014	valid_1's binary_logloss: 0.18844
--------------------------------------------------
 training fold 3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[112]	training's binary_logloss: 0.143689	valid_1's binary_logloss: 0.302855
--------------------------------------------------
 training fold 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[146]	training's binary_logloss: 0.11086	valid_1's binary_logloss: 0.341638
--------------------------------------------------
 training fold 5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[321]	training's binary_logloss: 0.0403105	valid_1's binary_logloss: 0.176041
--------------------------------------------------
 training fold 6
Training until validation scores 

/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/home/shirasukazushi/.local/lib/python3.10/site-packages/lightgbm/basic.

Early stopping, best iteration is:
[496]	training's binary_logloss: 0.0151238	valid_1's binary_logloss: 0.142935
--------------------------------------------------
 training fold 9
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[73]	training's binary_logloss: 0.191859	valid_1's binary_logloss: 0.314274
--------------------------------------------------
 training fold 10
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[175]	training's binary_logloss: 0.10164	valid_1's binary_logloss: 0.200518


In [34]:
best_params = study.best_trial.params
best_value = study.best_value

In [38]:
best_params["objective"] = "binary"
best_params["learning_rate"] = 0.01
best_params["metric"] = "binary_logloss"
best_params["verbosity"] = -1
best_params["boosting_type"] = "gbdt"

In [39]:
model = lgb.LGBMClassifier(**best_params)

In [40]:
model.fit(train_df[features], train_df[CFG.target_col])

[LightGBM] [Warning] bagging_fraction is set=0.47933199914345187, subsample=1.0 will be ignored. Current value: bagging_fraction=0.47933199914345187
[LightGBM] [Warning] lambda_l2 is set=4.17720537364469e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.17720537364469e-08
[LightGBM] [Warning] lambda_l1 is set=2.248432895125247e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.248432895125247e-08
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.583585138554347, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.583585138554347


LGBMClassifier(bagging_fraction=0.47933199914345187, bagging_freq=7,
               feature_fraction=0.583585138554347,
               lambda_l1=2.248432895125247e-08, lambda_l2=4.17720537364469e-08,
               learning_rate=0.01, metric='binary_logloss',
               min_child_samples=75, num_leaves=74, objective='binary',
               verbosity=-1)

In [47]:
prediction = model.predict_proba(test_df.drop(["Id", "Class"], axis=1))
submission = pd.DataFrame(columns = submission_df.columns)
submission['Id'] = test_df['Id']
submission[['class_0','class_1']] = prediction
submission.to_csv('submission.csv',index=False)

In [48]:
submission

,Id,class_0,class_1
0,00eed32682bb,0.811657,0.188343
1,010ebe33f668,0.811657,0.188343
2,02fa521e1838,0.811657,0.188343
3,040e15f562a2,0.811657,0.188343
4,046e85c7cc7f,0.811657,0.188343
